In [11]:
# windows打印本地摄像头列表
import cv2
import os
import numpy as np

# 打印摄像头列表
cam_list = []
for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        cam_list.append(i)
        print("摄像头{}:".format(i))
        print("  可用")





摄像头0:
  可用
摄像头1:
  可用
摄像头2:
  可用


In [12]:
# 使用本地摄像头
cap = cv2.VideoCapture(1)
while True:
    ret, frame = cap.read()
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [17]:
print(help(cv2.imdecode))

Help on built-in function imdecode:

imdecode(...)
    imdecode(buf, flags) -> retval
    .   @brief Reads an image from a buffer in memory.
    .   
    .   The function imdecode reads an image from the specified buffer in the memory. If the buffer is too short or
    .   contains invalid data, the function returns an empty matrix ( Mat::data==NULL ).
    .   
    .   See cv::imread for the list of supported formats and flags description.
    .   
    .   @note In the case of color images, the decoded images will have the channels stored in **B G R** order.
    .   @param buf Input array or vector of bytes.
    .   @param flags The same flags as in cv::imread, see cv::ImreadModes.

None


In [ ]:
import cv2
import numpy
import socket

# 创建UDP套接字
client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
client.connect(('127.0.0.1', 1031))

def main():
    try:
        client.sendto(b'START_OF_IMAGE', ('127.0.0.1', 1031))
        while True:
            rece_data = []
            while True:
                # 接收数据，拼接成完整的图片, 标志位为b'END_OF_IMAGE'
                data = client.recv(65535)
                if not data:
                    print("未接收到数据")
                    continue
                if b'END_OF_IMAGE' in data:
                    print("接收到结束标志")
                    break
                else:
                    rece_data.append(data)

            # 调试信息
            print("接收到的数据长度:", len(rece_data))
            if not rece_data:
                print("未接收到任何图像数据")
                continue

            client_data = b''.join(rece_data)  # 拼接成完整的图片
            # 将接收到的字节数据转换为numpy数组
            nparr = numpy.frombuffer(client_data, numpy.uint8)
            # decode image
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            if img is None:
                print("图像解码失败")
            else:
                cv2.imshow('server', img)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception as e:
        print(f"发生异常: {e}")
    finally:
        client.close()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()